## Session 3: Polymorphism, and more data types

### Types and polymorphism

OCaml is a *statically typed* programming language, i.e. everything has a type and the program will not compile unless its typing is correct. For example, `3 + 0.1` is allowed in Python, because Python will automatically convert `3` (an integer) to `3.0` (a float), and perform floating-point addition. In OCaml, you get a type error:

In [ ]:
3 + 0.1

The OCaml compiler knows the types of expressions by an algorithm called *type inference*. In `3 + 0.1`, it knows syntactically that `3` is a `int`, `+` is a function of type `int -> int -> int`, and `0.3` is a `float`. Since `float` is not `int`, it fails to match the type specification of `+`, and an error is produced.

Functions like `+` and `-` act on specific types only, but there are some functions that are applicable to all types:

In [ ]:
let duplicate x = (x, x)

OCaml gives this function type `'a -> 'a * 'a`. Here, `'a` is a *type variable*, meaning that the input can be of any type, and the return value would be a pair of data of the input type. This is called *polymorphism*, which is often desirable when we want to reduce code duplication. OCaml compiler infers the most general type to a definition, but we can also specify the types ourselves.

As an exercise, infer the types of the following functions. What operations do they perform?

In [ ]:
let id x = x

In [ ]:
let drop x y = x

In [ ]:
let swap f = (fun x y -> f y x)

In [ ]:
let compose g f = (fun x -> g (f x))

In [ ]:
let cur f = (fun x y -> f (x, y))

In [ ]:
let twice f = (fun x -> f (f x))

In [ ]:
let rec iter f n x =
  if n <= 0 then x
  else f (iter f (n - 1) x)

### Binary numbers

We can represent binary numbers as a sequence of zeros (`O`) and ones (`I`), with the left-most symbol being the **least** significant bit, and a special symbol `End` marking the end (or, should I say, start) of a sequence. For example, the binary number 1010 (which is 10 in decimal) is represented as `O (I (O (I End)))`.

We can now create a library of binary-number related functions, starting from the following two:
- `to_int : bin -> int` that converts a binary to an integer.
-  `from_int : int -> bin` that creates a binary from an integer.

Note that `from_int` is not tail-recursive. It does not matter in practice, because its space complexity is $O(log(i))$, and since OCaml's largest integer is $2^{63} - 1$, it won't overflow our stack with just 63 nested calls.

We put the least significant bit at the front because it is easier to implement arithmetic operations, such as increment and addition:
- `inc : bin -> bin` adds the binary number by one.
- `add : bin -> bin` adds up two binary numbers.

There are two situations with addition, whether the current bit has a carrying bit or not. We can define a helper function `add' : bool -> bin -> bin -> bin` that takes an extra parameter to indicate the carrying bit, but, a mutual-recursive definition is sometimes easier to write and to understand.

It would be fun to print these binary numbers as strings.
- `bin_to_string : bin -> string` turns a binary number to a string.

### Generating trees

In [ ]:
type 'a tree = Lf | Br of 'a * 'a tree * 'a tree

We learned how to represent trees and how to map and fold over a tree. Now we look at ways to generate int-valued trees with certain patterns.

- `ftree : int -> int -> int tree` generates a balanced tree with `k` on the top and `n` levels. For example, `ftree 1 3` gives a tree like `1; 2 3; 4 5 6 7`.
- `farr : int -> int tree` generates a functional array of size $2^n - 1$, where $n$ is the function argument. For example, `farr 3` gives a tree like `1; 2 4; 3 6 5 7`.

Key to generating these trees is to find the invariant between a node and its two children. For trees generated by `ftree`, the values of a node, its left child, and its right child satisfies $(n, l, r) = (k, 2k, 2k + 1)$. So, we give a recursive definition like the following:

Similarly, for trees generated by `farr`, a node and its two children satisfies $(n, l, r) = (k, k + s, k + 2s)$, where $s = 2^l$, $l$ is the node's level.